To crawl https://www.gov.sg/factually/page-2 then 
https://www.gov.sg/factually/content/are-more-foreigners-taking-away-jobs-of-singaporeans

reference https://www.digitalocean.com/community/tutorials/how-to-scrape-web-pages-with-beautiful-soup-and-python-3

Post activity thoughts:
The code should be written for robust-ness. I understand why json is one of the the preferred communciation format, and why dictionaries are useful. Even with missing information a dictionary-like file can still be created without hassle.

In [1]:
import requests
from bs4 import BeautifulSoup
import unicodedata
import numpy as np
import time
import pandas as pd

### Define function for `threading`

In [6]:
def get_articles_from_page(page_num, debug = False):
    '''
    given page_num
    returns category_name_texts, 
            category_name_links, 
            article_title_links,
            article_title_texts
    '''
    
    category_name_texts = []
#     category_name_links = []
    article_title_texts = []
    article_title_links = []
    
    if True:
#     try:
        page = requests.get('https://www.gov.sg/factually/page-{}'.format(page_num))
        soup = BeautifulSoup(page.text, 'html.parser')

#         soup = soup.find(class_="content factually-section")
        # narrow down to this section
#         print(soup)

        grid_items = soup.find_all(class_="news-info")
#         print(grid_items)
        for grid_item in grid_items:

#             try:
                category_item = grid_item.find(class_="category")
                category_name_text = category_item.text
                category_name_text = category_name_text.strip()
#                 category_name_link = category_item.get('href')

                article_title_item = grid_item.find('h2')
                article_title_text = article_title_item.text
                article_title_link = article_title_item.find('a').get('href')

                category_name_texts.append(category_name_text)
#                 category_name_links.append(category_name_link)
                article_title_texts.append(article_title_text)
                article_title_links.append(article_title_link)

#             except:
#                 print(page_num, "error")
                
        print(str(page_num) + " ", end="")

#     except:
#         print(page_num, "ERROR")
    
    return (category_name_texts, 
#             category_name_links, 
            article_title_texts,
            article_title_links)

In [9]:
# test one out
get_articles_from_page(30)

30 

(['Defence & Security', 'Education & Employment', 'Health', 'Housing'],
 ['When can I call an SCDF Ambulance?',
  'How can I remain employable?',
  "What's this MERS-CoV that’s been in the news lately?",
  'What HDB grants are there to help?'],
 ['/factually/content/when-can-i-call-an-scdf-ambulance',
  '/factually/content/how-can-i-remain-employable',
  '/factually/content/whats-this-merscov-thats-been-in-the-news-lately',
  '/factually/content/what-hdb-grants-are-there-to-help'])

### Crawling with `threading`

In [10]:
import urllib.request as urllib2
from multiprocessing.dummy import Pool as ThreadPool 
import time

start_time = time.time()

# make the Pool of workers
pool = ThreadPool(20) 

# open the urls in their own threads
# and return the results
results = pool.map(get_articles_from_page, range(1,51))  # CHANGE THIS

# close the pool and wait for the work to finish 
pool.close()
pool.join()

print(time.time() - start_time)

5 9 11 20 7 19 18 10 2 6 14 15 4 17 13 12 16 8 21 3 1 28 24 29 23 25 36 26 22 30 27 37 38 33 41 40 34 31 35 32 42 44 39 43 45 50 48 49 46 47 2.0202012062072754


### Process results

In [11]:
import numpy as np
print(np.shape(results))

category_name_texts = []
# category_name_links = []
article_title_texts = []
article_title_links = []

for result in results:
    category_name_texts += result[:][0]
#     category_name_links += result[:][1]
    article_title_texts += result[:][1]
    article_title_links += result[:][2]

links_crawled_2D_array = []

for a,b,c in zip(category_name_texts,
#                    category_name_links,
                   article_title_texts,
                   article_title_links):
    links_crawled_2D_array.append([a,b,c])

my_df = pd.DataFrame(links_crawled_2D_array)
my_df.to_csv('links_crawled.csv', index=False, header=["category_name_texts", 
#                                                        "category_name_links",
                                                       "article_title_texts",
                                                       "article_title_links"])

(50, 3)


In [14]:
links_crawled = pd.read_csv('links_crawled.csv', header=[0])
print(links_crawled.head())
print(links_crawled.tail())
# headlines = [headline for headline in articles["headline"][:]]

      category_name_texts                                article_title_texts  \
0  Education & Employment  Are more foreigners taking away jobs of Singap...   
1                  Health  Will the Government take back unused MediSave ...   
2             Environment  Can the prices in the 1962 Water Agreement be ...   
3  Education & Employment  MOM clarifies inaccuracies in TWC2’s articles ...   
4                  Health  Did the Government introduce CareShield Life t...   

                                 article_title_links  
0  /factually/content/are-more-foreigners-taking-...  
1  /factually/content/will-the-government-take-ba...  
2  /factually/content/can-the-prices-in-the-1962-...  
3      /factually/content/mom-clarifies-inaccuracies  
4  /factually/content/did-the-government-introduc...  
      category_name_texts                                article_title_texts  \
194      Law & Government  Is the Government's procurement process really...   
195     Economy & Finance  Wh

# Crawling for individual articles

In [15]:
links_crawled_matrix = [links_crawled.iloc[i] for i in range(len(links_crawled.index))]

In [16]:
# some usage
print(links_crawled.iloc[20])
# print(links_crawled.iloc[20].name)
# print(list(links_crawled["article_title_links"]))
# print(links_crawled.as_matrix)

category_name_texts                                      Savings & Taxes
article_title_texts    What are the taxes that the Government is cons...
article_title_links    /factually/content/what-are-the-taxes-that-the...
Name: 20, dtype: object


In [41]:
def get_info_from_link(link_crawled):
    article_link = 'https://www.gov.sg/' + link_crawled["article_title_links"]
    print(str(link_crawled.name) + " ", end="") # id actually

    if True:
#     try:
    #     print(article_link)
        page = requests.get(article_link)
        soup = BeautifulSoup(page.text, 'html.parser')
        print(soup)

        headline_item = soup.find(class_="title")
        headline = headline_item.contents[0]  
#         print(headline)

        date_item = soup.find(class_="op-modified")
        date = date_item.text
    #     date = date_item.select_one("span").text  # the same
#         print(date)

        article_text = ""
        text_soup = soup.find("article")
#         print(text_soup)
        text_paras = text_soup.select("p")  # searches <p>s, may have issues on some pages
        for text_para in text_paras[:-1]:
            text_line = text_para.text
            article_text += text_line
            article_text += "\n"
    #     print(article_text)

        article_text = unicodedata.normalize("NFKD", article_text)
        # https://stackoverflow.com/questions/10993612/python-removing-xa0-from-string

        if not headline or not date or not article_text:
            print("FAULTY", article_link, " " , str(link_crawled.name))

#         time.sleep(np.random.rand())
        

        
        return [link_crawled["category_name_texts"],
#                 link_crawled["category_name_links"],
                link_crawled["article_title_texts"],
                link_crawled["article_title_links"],
                headline,
                date,
                article_text]
        
#     except:
#         print(article_link)
#         return [link_crawled["category_name_texts"],
# #                 link_crawled["category_name_links"],
#                 link_crawled["article_title_texts"],
#                 link_crawled["article_title_links"],
#                 link_crawled["article_title_texts"],
#                 date,
#                 "Text not found."]

In [42]:
get_info_from_link(links_crawled_matrix[16])

16 <html><head>
<title>Access Denied</title>
</head><body>
<h1>Access Denied</h1>
 
You don't have permission to access "http://www.gov.sg/factually/content/is-it-true-our-public-universities-reserve-20-per-cent-of-their-places-for-foreign-students" on this server.<p>
Reference #18.8563c617.1537036883.af93fad
</p></body>
</html>



AttributeError: 'NoneType' object has no attribute 'contents'

### Crawl each individual link with `threading`

In [29]:
import urllib.request as urllib2
from multiprocessing.dummy import Pool as ThreadPool 
import time

start_time = time.time()

# make the Pool of workers
pool = ThreadPool(20) 

# open the urls in their own threads
# and return the results
results = pool.map(get_info_from_link, links_crawled_matrix[:])

# close the pool and wait for the work to finish 
pool.close()
pool.join()

print(time.time() - start_time)

0 3 6 9 12 15 18 21 24 27 30 33 36 39 42 45 48 51 54 57 13 34 7 | FAULTY https://www.gov.sg//factually/content/did-the-public-transport-council-quietly-axe-the-off-peak-pass-for-seniors 52 43   15 22 58  | 
37 55 16 46 4 25 49 40 1 35 19 31 28 10 | FAULTY https://www.gov.sg//factually/content/three-reasons-why-we-need-to-expand-our-airport-infrastructure  44  13  | 
14 8 56 53 23 5 26 59 | FAULTY https://www.gov.sg//factually/content/is-it-true-our-public-universities-reserve-20-per-cent-of-their-places-for-foreign-students   16  | 
17 38 50 47 60 2 63 11 20 29 41 66 69 72 75 | FAULTY https://www.gov.sg//factually/content/do-hdb-flat-buyers-own-their-flat   31  | 78 
32 81 84 87 90 | FAULTY https://www.gov.sg//factually/content/is-it-true-that-over-60-per-cent-of-foreign-domestic-workers-here-are-exploited   17 93 96 99  | 
102 61 64 70 73 105 108 111 76 114 85 117 79 67 | FAULTY82  https://www.gov.sg//factually/content/how-do-i-vote-on-11th-sept-when-i-am-overseas   90  | 
91 97 88 10

AttributeError: 'NoneType' object has no attribute 'contents'

In [20]:
# print(results)

In [43]:
import pandas as pd
my_df = pd.DataFrame(results)
my_df.to_csv('articles.csv', index=False, header=["category_name_texts", 
#                                                   "category_name_links",
                                                  "article_title_texts",
                                                  "article_title_links",
                                                  "headline",
                                                  "date",
                                                  "article_text"])

In [44]:
articles = pd.read_csv('articles.csv', header=[0])
print(articles.tail())

      category_name_texts                                article_title_texts  \
194      Law & Government  Is the Government's procurement process really...   
195     Economy & Finance  Why does inflation seem to hit me more than wh...   
196           Environment  Would an annual increase in rainfall cause mor...   
197  Transport & Motoring  Why is there a need for the Government to inve...   
198  Transport & Motoring  Why does the Government need to build the Nort...   

                                   article_title_links  \
194  /factually/content/is-the-governments-procurem...   
195  /factually/content/why-does-inflation-seem-to-...   
196  /factually/content/would-an-annual-increase-in...   
197  /factually/content/why-is-there-a-need-for-the...   
198  /factually/content/why-does-the-government-nee...   

                                              headline  \
194  \r\n                Is the Government's procur...   
195  \r\n                Why does inflation seem to...

In [ ]:
# okay it seems that I got blocked from spamming the site :/